## Import Library

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import torch.nn.functional as F
import os
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cpu


## Data Loader

In [3]:
data_dir = "/kaggle/input/gender-classification-dataset"

In [4]:
import os

# Cek semua folder yang ada di /kaggle/input
print("Daftar folder di /kaggle/input:")
print(os.listdir("/kaggle/input"))


Daftar folder di /kaggle/input:
['Validation', 'Training']


In [5]:
data_dir = "/kaggle/input"

print("Isi Dataset:")
print(os.listdir(data_dir))

print("\nIsi Training:")
print(os.listdir(os.path.join(data_dir, "Training")))

print("\nIsi Validation:")
print(os.listdir(os.path.join(data_dir, "Validation")))


Isi Dataset:
['Validation', 'Training']

Isi Training:
['female', 'male']

Isi Validation:
['female', 'male']


In [11]:
import os

data_dir = "/kaggle/input"

# Menampilkan isi folder utama
print("Isi Dataset:")
print(os.listdir(data_dir))

# Fungsi bantu untuk menghitung jumlah file per kelas
def hitung_jumlah_data(folder_path):
    total = 0
    for kelas in os.listdir(folder_path):
        kelas_path = os.path.join(folder_path, kelas)
        if os.path.isdir(kelas_path):
            jumlah_file = len(os.listdir(kelas_path))
            print(f"- {kelas}: {jumlah_file} file")
            total += jumlah_file
    print(f"Total file di folder {os.path.basename(folder_path)}: {total}\n")

# Training set
print("\nIsi Training:")
train_dir = os.path.join(data_dir, "Training")
print(os.listdir(train_dir))
print("\nJumlah data per kelas (Training):")
hitung_jumlah_data(train_dir)

# Validation set
print("\nIsi Validation:")
val_dir = os.path.join(data_dir, "Validation")
print(os.listdir(val_dir))
print("\nJumlah data per kelas (Validation):")
hitung_jumlah_data(val_dir)


Isi Dataset:
['Validation', 'Training']

Isi Training:
['female', 'male']

Jumlah data per kelas (Training):
- female: 23243 file
- male: 23766 file
Total file di folder Training: 47009


Isi Validation:
['female', 'male']

Jumlah data per kelas (Validation):
- female: 5841 file
- male: 5808 file
Total file di folder Validation: 11649



## Transformasi Gambar

In [6]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Path dataset langsung ke /kaggle/input
data_dir = "/kaggle/input"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],
                         [0.5, 0.5, 0.5])
])

# Gunakan nama folder sesuai hasil os.listdir() → "Training" dan "Validation"
train_data = datasets.ImageFolder(os.path.join(data_dir, 'Training'), transform=transform)
val_data = datasets.ImageFolder(os.path.join(data_dir, 'Validation'), transform=transform)

trainloader = DataLoader(train_data, batch_size=32, shuffle=True)
valloader = DataLoader(val_data, batch_size=32, shuffle=False)

print("Kelas terdeteksi:", train_data.classes)


Kelas terdeteksi: ['female', 'male']


## Model GoogleNet

In [7]:
# Define GoogleNet (InceptionV1) Architecture
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()
        
        # 1x1 branch
        self.branch1 = nn.Conv2d(in_channels, ch1x1, kernel_size=1)
        
        # 1x1 -> 3x3 branch
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, ch3x3red, kernel_size=1),
            nn.Conv2d(ch3x3red, ch3x3, kernel_size=3, padding=1)
        )
        
        # 1x1 -> 5x5 branch
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, ch5x5red, kernel_size=1),
            nn.Conv2d(ch5x5red, ch5x5, kernel_size=5, padding=2)
        )
        
        # 3x3 pool -> 1x1 branch
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, pool_proj, kernel_size=1)
        )
        
    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        
        outputs = [branch1, branch2, branch3, branch4]
        return torch.cat(outputs, 1)

class GoogleNet(nn.Module):
    def __init__(self, num_classes=2, aux_logits=True):
        super(GoogleNet, self).__init__()
        self.aux_logits = aux_logits
        
        # Initial layers
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=1)
        self.conv3 = nn.Conv2d(64, 192, kernel_size=3, padding=1)
        self.maxpool2 = nn.MaxPool2d(3, stride=2, padding=1)
        
        # Inception blocks
        self.inception3a = Inception(192, 64, 96, 128, 16, 32, 32)
        self.inception3b = Inception(256, 128, 128, 192, 32, 96, 64)
        self.maxpool3 = nn.MaxPool2d(3, stride=2, padding=1)
        
        self.inception4a = Inception(480, 192, 96, 208, 16, 48, 64)
        self.inception4b = Inception(512, 160, 112, 224, 24, 64, 64)
        self.inception4c = Inception(512, 128, 128, 256, 24, 64, 64)
        self.inception4d = Inception(512, 112, 144, 288, 32, 64, 64)
        self.inception4e = Inception(528, 256, 160, 320, 32, 128, 128)
        self.maxpool4 = nn.MaxPool2d(3, stride=2, padding=1)
        
        self.inception5a = Inception(832, 256, 160, 320, 32, 128, 128)
        self.inception5b = Inception(832, 384, 192, 384, 48, 128, 128)
        
        # Auxiliary classifiers
        if self.aux_logits:
            self.aux1 = InceptionAux(512, num_classes)
            self.aux2 = InceptionAux(528, num_classes)
        
        # Adaptive pooling and classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        # Initial layers
        x = F.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.maxpool2(x)
        
        # Inception blocks
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        
        x = self.inception4a(x)
        
        # Auxiliary classifier 1
        if self.aux_logits and self.training:
            aux1 = self.aux1(x)
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        
        # Auxiliary classifier 2
        if self.aux_logits and self.training:
            aux2 = self.aux2(x)
        
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        # Final layers
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        
        if self.aux_logits and self.training:
            return x, aux1, aux2
        return x

class InceptionAux(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(InceptionAux, self).__init__()
        self.conv = nn.Sequential(
            nn.AvgPool2d(kernel_size=5, stride=3),
            nn.Conv2d(in_channels, 128, kernel_size=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Sequential(
            nn.Linear(2048, 1024),  # Adjusted for 224x224 input
            nn.ReLU(inplace=True),
            nn.Dropout(0.7),
            nn.Linear(1024, num_classes)
        )
        
    def forward(self, x):
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Initialize model
model = GoogleNet(num_classes=2, aux_logits=True)
model = model.to(device)

## Loss and Optimizer

In [10]:

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Training and Validation

In [9]:
# Train the model
print("Starting training...")
model, train_losses, val_accuracies = train_model(
    model, trainloader, valloader, criterion, optimizer, scheduler, num_epochs=2
)

# Karena tidak ada folder Test, gunakan Validation sebagai test set
testloader = valloader

# Fungsi untuk menguji model
def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(testloader, desc='Testing'):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Load best model dan uji
print("Loading best model for testing...")
model.load_state_dict(torch.load('best_googlenet_model.pth'))
test_accuracy = test_model(model, testloader)

# Simpan model akhir
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'test_accuracy': test_accuracy,
    'classes': train_data.classes
}, 'final_googlenet_model.pth')

print("Training completed!")
print(f"Final Test Accuracy: {test_accuracy:.2f}%")


Starting training...


NameError: name 'train_model' is not defined

In [ ]:
# Validation phase
model.eval()
val_running_corrects = 0

with torch.no_grad():
    for inputs, labels in tqdm(valloader, desc='Validation'):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        val_running_corrects += torch.sum(preds == labels.data)

val_acc = val_running_corrects.double() / len(val_data)
val_accuracies.append(val_acc.cpu())

print(f'Val Acc: {val_acc:.4f}')

# Update learning rate
scheduler.step()

# Save best model
if val_acc > best_acc:
    best_acc = val_acc
    torch.save(model.state_dict(), 'best_googlenet_model.pth')
    print(f'New best model saved with accuracy: {best_acc:.4f}')

print()
print(f'Best val Acc: {best_acc:.4f}')

return model, train_losses, val_accuracies


In [ ]:
!pip install scikit-learn


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import torch
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(model, dataloader, classes):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Buat confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

    # Tampilkan
    plt.figure(figsize=(6, 6))
    disp.plot(cmap='Blues', values_format='d')
    plt.title("Confusion Matrix")
    plt.show()


In [ ]:
print("Menampilkan Confusion Matrix...")
plot_confusion_matrix(model, valloader, train_data.classes)